### URL parsing

In [190]:
import urllib.parse

# 한글 문자열
text = "강남역 맛집"

# URL 인코딩
encoded_text = urllib.parse.quote(text)

base_path=f"https://www.google.co.kr/maps/search/{encoded_text}"

print(encoded_text)

%EA%B0%95%EB%82%A8%EC%97%AD%20%EB%A7%9B%EC%A7%91


### googlemap_info

In [2]:
import selenium
from selenium.webdriver import Chrome
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.select import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.expected_conditions import visibility_of_all_elements_located,presence_of_element_located,presence_of_all_elements_located

# 레스토랑 db insert 기초 틀
def restuarant_properties(search_term,restaurant_name,url,address,rating,category,**kwrargs):
    
    properties={"검색어" : {"rich_text" : [{"text": {"content": search_term}}]},
                "음식점이름" : {"title" : [{"text" : {'content': restaurant_name}}]},
                "URL" : {"url" : url},
                "주소" : {"rich_text" : [{"text": {"content": address}}]},
                "별점" : {"number" : rating},
                "카테고리" : {"rich_text" : [{"text": {"content": category}}]}
                }
    return properties

# 리뷰 db insert 기초 틀
def review_properties(restaurant_name,user,review,**kwargs):

    properties={"음식점이름" : {"title" : [{"text": {"content": restaurant_name}}]},
                "유저" : {"rich_text" : [{"text": {"content": user}}]},
                "리뷰" : {"rich_text" : [{"text": {"content": review}}]}
                }
    return properties


info_num=20

Main_Drive=Chrome()
Main_Drive.get(base_path)
Sub_Drive=Chrome()

main_box=WebDriverWait(Main_Drive,3).until(
    presence_of_element_located([By.XPATH, '//*[@role="feed"]']))

while True:
    main_box.send_keys(Keys.PAGE_DOWN)

    feed_box=WebDriverWait(Main_Drive,3).until(
        visibility_of_all_elements_located([By.CLASS_NAME,"hfpxzc"]))
    
    if len(feed_box)>info_num:
        break

infos=[]

for sub_feed in feed_box:
    search_term=text #위의 코드 에서 가져옴 수정필요
    restaurant_name=sub_feed.get_attribute("aria-label")
    url=sub_feed.get_attribute("href")

    infos.append(
        restuarant_properties(search_term,restaurant_name,url,None,None,None)
    )

for sub_info in infos:
    Sub_Drive.get(sub_info["URL"])

    #별점
    rating=WebDriverWait(Sub_Drive,3).until(
    presence_of_element_located([By.XPATH, '//*[@role="img"]'])).get_attribute("aria-label")
    # 전처리해야함
    rating="".join([term for term in rating if not term.isalpha()]).strip()
    rating=float(rating)

    #음식점 카테고리
    category=WebDriverWait(Sub_Drive,3).until(
    presence_of_element_located([By.CSS_SELECTOR,'button.DkEaL'])).text

    #정보선택칸
    tabs=WebDriverWait(Sub_Drive,3).until(
    presence_of_all_elements_located([By.XPATH, '//*[@role="tab"]']))
    tab_select={element.text:element for element in tabs}
    
    #하위 정보칸
    info_region=WebDriverWait(Sub_Drive,3).until(
    presence_of_element_located([By.XPATH, '//*[@role="region"]']))
    #주소
    address=WebDriverWait(info_region,3).until(
    presence_of_element_located([By.XPATH, '//*[@data-item-id="address"]']))
    

/Users/ijeonghun/Documents/github/GoogleMap/googlemap/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [105]:
Main_Drive=Chrome()
Main_Drive.get(base_path)
info_num=30
#Sub_Drive=Chrome()

main_box=WebDriverWait(Main_Drive,3).until(
    presence_of_element_located([By.XPATH, '//*[@role="feed"]']))

while True:
    main_box.send_keys(Keys.PAGE_DOWN)

    feed_box=WebDriverWait(Main_Drive,3).until(
        visibility_of_all_elements_located([By.CSS_SELECTOR,"a.hfpxzc"]))
    
    if len(feed_box)>info_num:
        break

infos=[]

for sub_feed in feed_box:
    search_term=text #위의 코드 에서 가져옴 수정필요
    restaurant_name=sub_feed.get_attribute("aria-label")
    url=sub_feed.get_attribute("href")

    infos.append(
        restuarant_properties(search_term,restaurant_name,url,None,None,None)
    )

In [107]:
Main_Drive.quit()

In [135]:
from threading import Thread

In [750]:
%%writefile GooglemapCraw.py
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.remote.webelement import WebElement
import urllib.parse
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.expected_conditions import visibility_of_all_elements_located,presence_of_element_located,presence_of_all_elements_located
from threading import Thread
from typing import List,Dict

class GooglemapCraw():
    def  __init__(self,search_term,craw_num:int,review_num:int=100,thread_num:int=1,optional:bool=True):

        # Chrome 옵션 설정
        self.chrome_options = Options()
        if optional:
            self.chrome_options.add_argument("--headless")  # Headless 모드
        else:
            pass

        self.main_drive=Chrome(options=self.chrome_options) # url까지 크롤
        self.craw_num=craw_num
        self.search_term=search_term
        self.infos=[] # 레스토랑 저장공간
        self.reviews=[] #리뷰 저장공간
        self.thread_num=thread_num
        self.review_num=review_num

        encoded_text = urllib.parse.quote(self.search_term)
        self.base_path=f"https://www.google.co.kr/maps/search/{encoded_text}" # 크롤링이 될 주소(파싱될 예정)
        pass

    def __restuarant_properties(self,search_term,restaurant_name,url,address,rating,category,**kwrargs):
    
        properties={"검색어" : {"rich_text" : [{"text": {"content": search_term}}]},
                "음식점이름" : {"title" : [{"text" : {'content': restaurant_name}}]},
                "URL" : {"url" : url},
                "주소" : {"rich_text" : [{"text": {"content": address}}]},
                "별점" : {"number" : rating},
                "카테고리" : {"rich_text" : [{"text": {"content": category}}]}
                }
        return properties

    # 리뷰 db insert 기초 틀
    def __review_properties(self,restaurant_name,user,review,**kwargs):

        properties={"음식점이름" : {"title" : [{"text": {"content": restaurant_name}}]},
                    "유저" : {"rich_text" : [{"text": {"content": user}}]},
                    "리뷰" : {"rich_text" : [{"text": {"content": review}}]}
                    }
        return properties
    
    # 멀티 스래딩 적용 
    def __crawras(self,sub_info_list:List[Dict],**kwargs):
        sub_drive=Chrome(options=self.chrome_options)
        for sub_info in sub_info_list:
            sub_drive.get(sub_info["url"])

            #별점
            rating=WebDriverWait(sub_drive,3).until(
            presence_of_element_located([By.XPATH, '//*[@role="img"]'])).get_attribute("aria-label")
            #전처리
            rating="".join([term for term in rating if not term.isalpha()]).strip()
            rating=float(rating)

            #음식점 카테고리
            category=WebDriverWait(sub_drive,3).until(
            presence_of_element_located([By.XPATH,"//button[contains(@jsaction,'category')]"])).text
            
            #하위 정보칸
            info_region=WebDriverWait(sub_drive,3).until(
            presence_of_element_located([By.XPATH, '//*[@role="region"]']))
            #주소
            address=WebDriverWait(info_region,3).until(
            presence_of_element_located([By.XPATH,'//button[@data-item-id="address"]'])).get_attribute("aria-label").replace("주소 :","").strip()

            sub_info.update(
                {"address":address,
                "rating":rating,
                "category":category}
            )
        sub_drive.quit()
    
    # 구글 리뷰 선택 평가(표의 형태로 변동성이 큰 리뷰 틀)
    def __subrevtable(self,sub_review:WebElement):
        splited_table={}
        try:
            reviewer_table=WebDriverWait(sub_review,3).until(
                    presence_of_all_elements_located([By.XPATH,".//div[@jslog]/div[contains(@jslog,'metadata')]"]))
            #구분자 설정
            def spliter(text:str):
                split_txt="\n" if "\n" in text else ":"
                return text.split(split_txt)
            for row in reviewer_table:
                splited_table.update({key:value for key,value in [spliter(row.text)]})
            
        except:
            pass

        return splited_table


    # 리뷰 크롤링 작업
    def __crawrev(self,sub_info_list:List[Dict],**kwargs):
        sub_drive=Chrome(options=self.chrome_options)
        for sub_info in sub_info_list:
            sub_drive.get(sub_info["url"])

            #정보선택칸
            tabs=WebDriverWait(sub_drive,3).until(
            presence_of_all_elements_located([By.XPATH, '//*[@role="tab"]']))
            tab_select={element.text.strip():element for element in tabs} 

            #리뷰 크롤링
            tab_select["리뷰"].click() #리뷰창으로 전환

            #현재 리뷰 개수
            total_review_num=WebDriverWait(sub_drive,3).until(
            presence_of_element_located([By.XPATH,"//div[contains(text(), '리뷰') and contains(@jslog,'metadata')]"])).text
            #전처리
            total_review_num=int("".join([term for term in total_review_num if term.isnumeric()]))

            #스크롤가능한 바디찾기
            candidate_list=WebDriverWait(sub_drive,3).until(
                 presence_of_all_elements_located([By.XPATH,"//div[contains(@jslog,'mutable')]"]))
            try:
                scroll_body=[element for element in candidate_list 
                         if any(["auto" in element.value_of_css_property("overflow"),
                                 "scroll" in element.value_of_css_property("overflow")])][0]
            
                while True: 
                    scroll_body.send_keys(Keys.PAGE_DOWN)

                    #리뷰 각각의 객체
                    review_box=WebDriverWait(sub_drive,3).until(
                        presence_of_all_elements_located([By.XPATH
                        ,"//div[@data-review-id and contains(@jsaction,'review.out')]"])) 

                    if (len(review_box)==total_review_num)or(len(review_box)>self.review_num):
                        break
            except:
                review_box=WebDriverWait(sub_drive,3).until(
                        presence_of_all_elements_located([By.XPATH
                        ,"//div[@data-review-id and contains(@jsaction,'review.out')]"]))
            # 리뷰 돌아가며 더보기 누르기(공통 작업)
            for sub_review in review_box:
                base_dict=sub_info.copy()
                sub_drive.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", sub_review)

                try:
                    button=WebDriverWait(sub_review,3).until(
                    presence_of_element_located([By.XPATH,".//button[contains(text(),'자세히')]"]))
                    button.click()
                except: # 리뷰가 짧으면 없음
                    pass

                #리뷰 유저 정보(이름, 주소)
                reviewer_info=WebDriverWait(sub_review,3).until(
                    presence_of_element_located([By.XPATH,".//button[@aria-label and contains(@jsaction,'reviewerLink')]"]))
                
                reviewer_name=reviewer_info.get_attribute("aria-label").replace("사진","")
                reviewer_href=reviewer_info.get_attribute("data-href")

                # 총합 별점
                reviewer_total_rating=WebDriverWait(sub_review,3).until(
                    presence_of_element_located([By.XPATH
                    ,".//span[@role='img' and contains(@aria-label,'별표')]"])).get_attribute("aria-label")
                reviewer_total_rating=float("".join([temp for temp in reviewer_total_rating if not temp.isalpha()]))

                #리뷰 본문
                reviewer_rev=WebDriverWait(sub_review,3).until(
                    presence_of_element_located([By.XPATH,".//div[@lang='ko']/span[not(@jslog)]"])).text

                #하위 리뷰 table 구조화
                rev_sub_table=self.__subrevtable(sub_review)

                base_dict.update(
                    {"user":reviewer_name,
                     "user_url":reviewer_href,
                     "review_total_rate":reviewer_total_rating,
                     "review_text":reviewer_rev,}
                )
                base_dict.update(rev_sub_table)

                self.reviews.append(base_dict)
                        
        sub_drive.quit()
        #return sub_info
    
    def craw_ras(self): # 기초정보 수집 -> 1개의 스래딩으로 충분
        self.main_drive.get(self.base_path)
        # 스크롤을 내리는 공간
        main_box=WebDriverWait(self.main_drive,3).until(
        presence_of_element_located([By.XPATH, '//*[@role="feed"]']))

        while True:
            main_box.send_keys(Keys.PAGE_DOWN)

            feed_box=WebDriverWait(self.main_drive,3).until(
                 presence_of_all_elements_located([By.XPATH,"//a[@href and @aria-label]"]))
            

            if len(feed_box)>self.craw_num:
                break
        
        for sub_feed in feed_box:
            #음식점 이름
            restaurant_name=sub_feed.get_attribute("aria-label")
            url=sub_feed.get_attribute("href")

            self.infos.append(
                {"search_term":self.search_term,
                 "restaurant_name":restaurant_name,
                 "url":url}
            )

        self.main_drive.quit()

        thread_list4ras=[]
        m=len(self.infos)//self.thread_num
        for n in range(self.thread_num):
            kw=self.infos[n*m:(n+1)*m]
            sub_thread4ras=Thread(target=self.__crawras,kwargs=({"sub_info_list":kw}))
            thread_list4ras.append(sub_thread4ras)
            sub_thread4ras.start()        
                
        for sub_thread in thread_list4ras:
            sub_thread.join()

    def craw_rev(self):
        thread_list4rev=[]
        for n in range(self.thread_num):
            kw=self.infos[n*m:(n+1)*m]
            sub_thread4rev=Thread(target=self.__crawrev,kwargs=({"sub_info_list":kw}))
            thread_list4rev.append(sub_thread4rev)
            sub_thread4rev.start()

        for sub_thread in thread_list4rev:
            sub_thread.join()



Writing GooglemapCraw.py


In [744]:
test_class=GooglemapCraw("강남역 맛집",20,10,2,False)

In [745]:
test_class.craw_ras()

In [746]:
test_class.craw_rev()

Exception in thread Thread-2922:
Traceback (most recent call last):
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/ijeonghun/Documents/github/GoogleMap/googlemap/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/bv/d7dzvp616v1d6dz4vbkn7zw40000gn/T/ipykernel_5711/2287795330.py", line 110, in __crawrev
  File "/Users/ijeonghun/Documents/github/GoogleMap/googlemap/lib/python3.9/site-packages/selenium/webdriver/support/wait.py", line 105, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 



In [749]:
from pandas import DataFrame
DataFrame(test_class.reviews)

,search_term,restaurant_name,url,address,rating,category,user,user_url,review_total_rate,review_text,서비스,식사 유형,1인당 가격,음식,분위기,추천 요리,유아 동반 환경,주차 공간,주차 옵션,주차,소음 수준,대기 시간
0,강남역 맛집,바비레드 강남본점,https://www.google.co.kr/maps/place/%EB%B0%94%...,"주소: 서울특별시 강남구 봉은사로6길 39, 지하 바비레드",4.3,퓨전 음식점,Zion Y,https://www.google.com/maps/contrib/1144267992...,2.0,주말 저녁에도 빈 자리가 많다.\n인근에 비해선 싸지만 그래도 부담스런 가격이다.\...,2,저녁 식사,"₩10,000~20,000",3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,강남역 맛집,바비레드 강남본점,https://www.google.co.kr/maps/place/%EB%B0%94%...,"주소: 서울특별시 강남구 봉은사로6길 39, 지하 바비레드",4.3,퓨전 음식점,SANGYON LEE,https://www.google.com/maps/contrib/1039646889...,5.0,"화이트 칼라의 내부에 커다란 샹그리아가 반겨주고, 밝은 모습으로 반겨주는 직원들.....",5,점심 식사,"₩20,000~30,000",5,4,스테이크 샐러드,NaN,NaN,NaN,NaN,NaN,NaN
2,강남역 맛집,바비레드 강남본점,https://www.google.co.kr/maps/place/%EB%B0%94%...,"주소: 서울특별시 강남구 봉은사로6길 39, 지하 바비레드",4.3,퓨전 음식점,노시경TV - Earth Traveller,https://www.google.com/maps/contrib/1090239660...,3.0,가성비는 좋은 편이나 음식의 맛은 평범합니다. 바비 스테이크의 고기는 육즙이 많지 ...,4,저녁 식사,"₩10,000~20,000",3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,강남역 맛집,바비레드 강남본점,https://www.google.co.kr/maps/place/%EB%B0%94%...,"주소: 서울특별시 강남구 봉은사로6길 39, 지하 바비레드",4.3,퓨전 음식점,문지완,https://www.google.com/maps/contrib/1131087201...,5.0,여자친구랑 데이트로 왔는데 생각보다 너무 맛있고 분위기도 좋아서 만족스럽습니다 :)...,5,저녁 식사,"₩60,000~70,000",5,5,"레드크림스튜, 레드밥추가, 스테이크 샐러드",NaN,NaN,NaN,NaN,NaN,NaN
4,강남역 맛집,바비레드 강남본점,https://www.google.co.kr/maps/place/%EB%B0%94%...,"주소: 서울특별시 강남구 봉은사로6길 39, 지하 바비레드",4.3,퓨전 음식점,food_dairy bang,https://www.google.com/maps/contrib/1067809902...,4.0,한식과 조화를 이루는 파스타가 매력적이였고 레드밥이랑 파스타에 비벼 먹는 것도 아주...,4,저녁 식사,"₩20,000~30,000",4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,강남역 맛집,무월식탁 강남본점,https://www.google.co.kr/maps/place/%EB%AC%B4%...,주소: 서울특별시 강남구 강남대로102길 23,3.8,음식점,Sarah Y,https://www.google.com/maps/contrib/1149391605...,4.0,분위기가 좋습니다. 한식집이나 분위기는 트렌디한 외국 펍 느낌입니다. 가격도 합리적...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
396,강남역 맛집,무월식탁 강남본점,https://www.google.co.kr/maps/place/%EB%AC%B4%...,주소: 서울특별시 강남구 강남대로102길 23,3.8,음식점,Jinhee Kim,https://www.google.com/maps/contrib/1181858669...,4.0,깔끔하고 맛있어요.\n가격도 적당합니다.\n단품으로 추가할 수있는것도 너무 좋아요.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
397,강남역 맛집,무월식탁 강남본점,https://www.google.co.kr/maps/place/%EB%AC%B4%...,주소: 서울특별시 강남구 강남대로102길 23,3.8,음식점,원모아,https://www.google.com/maps/contrib/1111851082...,5.0,정갈하게 한 쟁반씩이라서 깔끔하게 먹기좋았어요~ 양도 적당하고요~! 대신 밥이 좀 ...,5,저녁 식사,"₩10,000~20,000",4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
398,강남역 맛집,무월식탁 강남본점,https://www.google.co.kr/maps/place/%EB%AC%B4%...,주소: 서울특별시 강남구 강남대로102길 23,3.8,음식점,다뚜Daddoo,https://www.google.com/maps/contrib/1131021988...,4.0,"평소에 가면 항상 웨이팅 가득해서 적어도 4,5팀이 문앞에서 서성이는 것을 목격할 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [722]:
test_class.crawrev1(test_infos)

In [681]:
test_class.infos[19]["url"]

'https://www.google.co.kr/maps/place/%EA%B0%95%EB%82%A8%EC%A7%84%ED%95%B4%EC%9E%A5/data=!4m7!3m6!1s0x357ca1582a882a4b:0x6fa9110f4c3e1afd!8m2!3d37.4994553!4d127.0292471!16s%2Fg%2F11g8ljsxpg!19sChIJSyqIKlihfDUR_Ro-TA8RqW8?authuser=0&hl=ko&rclk=1'

In [725]:
test_class.reviews

[{'search_term': '강남역 맛집',
  'restaurant_name': '바비레드 강남본점',
  'url': 'https://www.google.co.kr/maps/place/%EB%B0%94%EB%B9%84%EB%A0%88%EB%93%9C+%EA%B0%95%EB%82%A8%EB%B3%B8%EC%A0%90/data=!4m7!3m6!1s0x357ca3e29805ba2f:0x339ec75093d60a3b!8m2!3d37.5026295!4d127.0278967!16s%2Fg%2F11bbtlp9qn!19sChIJL7oFmOKjfDUROwrWk1DHnjM?authuser=0&hl=ko&rclk=1',
  'address': '\ue0c8\n서울특별시 강남구 봉은사로6길 39, 지하 바비레드',
  'rating': 4.3,
  'category': '퓨전 음식점',
  'user': 'Zion Y ',
  'user_url': 'https://www.google.com/maps/contrib/114426799206876262511/reviews?hl=ko',
  'review_text': '주말 저녁에도 빈 자리가 많다.\n인근에 비해선 싸지만 그래도 부담스런 가격이다.\n파스타 19000원.\n주문 후 음식이 나오기까지 15분 이상 걸리는 것 같다.\n테이블 간 거리가 가까워 대화가 들린다.\n너무 밝다. 은은한 조명이거나 채도가 좀 낮으면 좋겠다. 미술관이 아닌 양식집인데 분위기가 좀 나야지..\n파스타 소스에 밥 비벼먹으라 빨간 밥을 준다. 그래서 바비레드. 쌀품종이 빨간색이라고..',
  '서비스': ' 2',
  '식사 유형': '저녁 식사',
  '1인당 가격': '₩10,000~20,000',
  '음식': ' 3',
  '분위기': ' 2'},
 {'search_term': '강남역 맛집',
  'restaurant_name': '바비레드 강남본점',
  'url': 'https://www.google.co.kr/maps/place/%EB

In [126]:
test_class.close()

In [726]:
test_driver=Chrome()
test_driver.get(test_class.infos[19]["url"])
#test_driver.get(base_path)

------

In [737]:
address=WebDriverWait(test_driver,3).until(
            presence_of_element_located([By.XPATH, '//button[@data-item-id="address"]']))

In [739]:
address.get_attribute("aria-label")

'주소: 서울특별시 강남구 강남대로102길 23 '

In [743]:
test_driver.quit()

In [683]:
review_box=WebDriverWait(test_driver,3).until(
                        presence_of_all_elements_located([By.XPATH
                        ,"//div[@data-review-id and contains(@jsaction,'review.out')]"]))

In [703]:
def subrevtable(sub_review:WebElement):
    splited_table={}
    try:
        reviewer_table=WebDriverWait(sub_review,3).until(
                presence_of_all_elements_located([By.XPATH,".//div[@jslog]/div[contains(@jslog,'metadata')]"]))
        #구분자 설정
        def spliter(text:str):
            split_txt="\n" if "\n" in text else ":"
            return text.split(split_txt)
        for row in reviewer_table:
            splited_table.update({key:value for key,value in [spliter(row.text)]})
        return splited_table
    except:
        return splited_table

In [708]:
reviewer_table=WebDriverWait(review_box[3],3).until(
                presence_of_all_elements_located([By.XPATH,".//div[@jslog]/div[contains(@jslog,'metadata')]"]))

TimeoutException: Message: 


In [707]:
subrevtable(review_box[3])

{}

In [671]:
test_driver.quit()

In [535]:
total_review_num=WebDriverWait(test_driver,3).until(
            presence_of_element_located([By.XPATH,"//div[contains(text(), '리뷰') and contains(@jslog,'metadata')]"]))

In [593]:
#스크롤가능한 바디찾기
candidate_list=WebDriverWait(test_driver,3).until(
                 presence_of_all_elements_located([By.XPATH,"//div[@tabindex='-1']"]))

scroll_body=[element for element in candidate_list 
                         if any(["auto" in element.value_of_css_property("overflow"),
                                 "scroll" in element.value_of_css_property("overflow")])]

In [636]:
candidate_list=WebDriverWait(test_driver,3).until(presence_of_all_elements_located([By.XPATH,"//div[contains(@jslog,'mutable')]"]))

In [638]:
[x.value_of_css_property("overflow") for x in candidate_list]

['visible', 'visible', 'visible', 'visible', 'visible', 'hidden auto']

In [633]:
candidate_list[1].value_of_css_property("overflow")

'hidden'

In [620]:
candidate_list[5].value_of_css_property("overflow")

'hidden auto'

In [596]:
scroll_body[0].get_attribute("jslog")

'26354;mutable:true;'

In [592]:
[x.get_attribute("jslog") for x in candidate_list]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 '26354;mutable:true;',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 '180487',
 None,
 None]

In [583]:
[x.get_attribute("jslog") for x in candidate_list]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 '26354;mutable:true;',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 '180487',
 None,
 None]

In [557]:
test_driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [552]:
scroll_body.send_keys(Keys.PAGE_DOWN)

In [538]:
test_driver.quit()

In [328]:
feed_box=WebDriverWait(test_driver,3).until(
                 presence_of_all_elements_located([By.XPATH,"//div[@tabindex='-1']"]))

In [329]:
for feed in feed_box:
    print(feed.value_of_css_property("overflow"))
    if "auto" in feed.value_of_css_property("overflow"):
        break

hidden
hidden
visible
visible
visible
visible
visible
visible
hidden
visible
hidden
hidden auto


In [330]:
feed.send_keys(Keys.PAGE_DOWN)

In [421]:
test_box=WebDriverWait(feed,3).until(
                 presence_of_all_elements_located([By.XPATH
                                                   ,"//div[@data-review-id and contains(@jsaction,'review.out')]"]))

In [432]:
test_driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", test_box[3])

In [ ]:
but_test=WebDriverWait(test_box[10],3).until(
                    presence_of_element_located([By.XPATH,".//button[contains(text(),'자세히')]"]))

In [378]:
but_test.click()

In [425]:
reviewer_info=WebDriverWait(test_box[10],3).until(
                    presence_of_element_located([By.XPATH,".//button[@aria-label and contains(@jsaction,'reviewerLink')]"]))

In [426]:
reviewer_rating=WebDriverWait(test_box[10],3).until(
                    presence_of_element_located([By.XPATH,".//span[@role='img' and contains(@aria-label,'별표')]"]))

In [427]:
reviewer_rating.get_attribute("aria-label")

'별표 4개'

In [429]:
reviewer_rev=WebDriverWait(test_box[10],3).until(
                    presence_of_element_located([By.XPATH,".//div[@lang='ko']/span[not(@jslog)]"])).text

In [430]:
print(reviewer_rev)

브릭오븐은 아직도 예전의 맛을 유지하고 있는것같다. 정통 미국식 피자를 변함없이 즐길 수있다. 양도 크고 배도 부르지만.


In [444]:
reviewer_table=WebDriverWait(test_box[3],3).until(
                    presence_of_all_elements_located([By.XPATH,".//div[@jslog]/div[contains(@jslog,'metadata')]"]))

In [464]:
reviewer_table[0].text

'서비스\n매장 내 식사'

In [462]:
ros[4].text

''

In [255]:
test_driver.quit()

In [171]:
tab_select["리뷰"].click()

In [181]:
test_driver.find_element(By.XPATH,"//div[contains(text(), '리뷰')]")

<selenium.webdriver.remote.webelement.WebElement (session="3b2fad94135727537c2907b051d73ed4", element="f.8B8818B3D0ED11F3C24611D3C1BDB109.d.BBD111B9F2019B07A2B31414B0EE70AE.e.143")>

In [180]:
aa='리뷰 1,167개'
[t for t in aa if t.isdigit()]

['1', '1', '6', '7']

In [ ]:
get_rating=test_driver.find_element(By.CLASS_NAME,"ceNzKf").get_attribute("aria-label")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".jANrlb>ceNzKf"}
  (Session info: chrome=131.0.6778.205); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x000000010504f184 cxxbridge1$str$ptr + 3626716
1   chromedriver                        0x00000001050479d4 cxxbridge1$str$ptr + 3596076
2   chromedriver                        0x0000000104ab4968 cxxbridge1$string$len + 89228
3   chromedriver                        0x0000000104af8d50 cxxbridge1$string$len + 368756
4   chromedriver                        0x0000000104b325b4 cxxbridge1$string$len + 604376
5   chromedriver                        0x0000000104aed568 cxxbridge1$string$len + 321676
6   chromedriver                        0x0000000104aee1b8 cxxbridge1$string$len + 324828
7   chromedriver                        0x00000

In [16]:
take_img=WebDriverWait(test_driver,3).until(
    presence_of_element_located([By.XPATH, '//*[@role="img"]']))

In [41]:
res_info=WebDriverWait(test_driver,3).until(
    presence_of_element_located([By.XPATH, '//*[@role="region"]']))

In [ ]:
address=WebDriverWait(res_info,3).until(
    presence_of_element_located([By.XPATH, '//*[@data-item-id="address"]']))

In [45]:
address.get_attribute("aria-label")

'주소: 서울특별시 강남구 강남대로96길 9 '

In [50]:
tabs=WebDriverWait(test_driver,3).until(
    presence_of_element_located([By.XPATH, '//*[@role="tab"]']))

In [52]:
tabs.

'개요'

In [53]:
test_driver.quit()

In [3]:
Main_Drive.quit()

### Restaurant_info

In [88]:
feed_box[0].get_attribute("aria-label")

'바비레드 강남본점'

In [78]:
main_box.get_attribute("role")

'feed'

In [72]:
feed_box[0].get_attribute("aria-label")

'바비레드 강남본점'

In [89]:
Main_Drive.quit()

In [59]:
test_elements=Main_Drive.find_elements(By.CLASS_NAME,"hfpxzc")

MaxRetryError: HTTPConnectionPool(host='localhost', port=57278): Max retries exceeded with url: /session/099672721e87bc05898169d8f5370121/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x109ffabb0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [47]:
test_elements[-1].get_attribute("aria-label")

'땀땀'

In [48]:
test_elements[-1].get_attribute("href")

'https://www.google.co.kr/maps/place/%EB%95%80%EB%95%80/data=!4m7!3m6!1s0x357cbba6cab8be71:0x123ebf3924dc0759!8m2!3d37.5003867!4d127.027994!16s%2Fg%2F11hcqrdf1p!19sChIJcb64yqa7fDURWQfcJDm_PhI?authuser=0&hl=ko&rclk=1'

In [49]:
Main_Drive.quit()

In [22]:
# 검색창
feedbox=Main_Drive.find_elements(By.TAG_NAME,"div")

In [25]:
feed_test=[element for element in feedbox if element.get_attribute("role")=="feed"]

In [29]:
feed_test[0].get_attribute("aria-label")

'강남역 맛집에 대한 검색결과'

In [33]:
from selenium.webdriver.common.keys import Keys

for _ in range(10):
    feed_test[0].send_keys(Keys.PAGE_DOWN)

테스트 끝.

In [37]:
Main_Drive.quit()

In [40]:
from threading import Thread

구조 

step 1 : url get

step 2 : url을 통해 세부정보 get